In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin, urlparse

def crawl_and_fetch_text(base_url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

        # Set of visited URLs to avoid duplicates
        visited = set()
        text_data = {}

        # Fetch the base page
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, "html.parser")

        # Add the base URL to visited and fetch its text
        visited.add(base_url)
        text_data[base_url] = soup.get_text(separator="\n", strip=True)

        # Find all anchor tags with href attributes
        links = soup.find_all("a", href=True)
        for link in links:
            # Resolve relative URLs
            href = urljoin(base_url, link['href'])

            # Skip if already visited or if the link is external
            if href not in visited and urlparse(href).netloc == urlparse(base_url).netloc:
                try:
                    # Fetch and process the page
                    page_response = requests.get(href, headers=headers)
                    page_response.encoding = page_response.apparent_encoding
                    page_soup = BeautifulSoup(page_response.text, "html.parser")
                    page_soup = BeautifulSoup(page_response.text, "html.parser")

                    # Extract text and store it
                    text_data[href] = page_soup.get_text(separator="\n", strip=True)
                    visited.add(href)
                except requests.exceptions.RequestException as e:
                    print(f"Failed to fetch {href}: {e}")

        return text_data

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return {}

# Example usage
base_url = "http://skolkarybicka.cz"
text_by_page = crawl_and_fetch_text(base_url)

# # Print results (text from each page)
# for url, text in text_by_page.items():
#     print(f"URL: {url}\n")
#     print(f"Text: {text[:500]}...\n")  # Print first 500 characters of each page's text


In [2]:
text_by_page

{'http://skolkarybicka.cz': 'Úvod | Školka Rybička\nÚvod\nProgram školky\nProvoz\nFotogalerie\nKdo jsme\nNapsali o nás\nKontakt\nAktuality\nAktuality\n20\nPRO\n2024\nZájmové kroužky a aktivity v Rybičce leden - květen 2025\n9\nZÁŘ\n2024\nPrávě vypsány zájmové kroužky v Rybičce\n5\nZÁŘ\n2024\nPrvní třídní schůzka 3. září 2024\nzápis ze schůzky\nArchiv aktualit\nPřírodovědecká fakulta Univerzity Karlovy\nVítejte na našich stránkách\nŠkolní rok 2024/ 2025\nNejbližší akce školky:\n6.2. -\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Návštěva Toulcova dvora (Koníci)\n10. - 14.2.\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 jarní prázdniny v Rybičce\n25.2. -\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Masopust\n1.4. -\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 jóga se Sítou\n17.4. -\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Zelený čtvrtek zkrácený provoz\n24.4.\xa0\xa0\xa0\xa0\xa0\

Split to sentences and use regex to extract the value.

In [3]:
import spacy
import re

def get_value_from_text(text_content):
    # Load SpaCy model for Czech (use stanza or similar library if SpaCy's model isn't available)
    # This requires a Czech language model, such as `spacy-stanza` or `cz-spacy-model`.
    nlp = spacy.blank("cs")  # Replace with `spacy.load("cs")` if Czech model is available.
    nlp.add_pipe('sentencizer')

    # Process the text
    doc = nlp(text_content)

    sentences = [sent.text for sent in doc.sents]

    # Filter sentences that contain the word "školné"
    relevant_sentences = [sentence for sentence in sentences if "měsíčně" in sentence.lower()]

    # Define a regex pattern to extract the fee (number followed by "Kč")
    fee_pattern = r"(?<!\d/)(\b\d{1,3}(?:[.,\s]\d{3})*(?:,-)?)\s*(?:Kč)?\b(?!/\d)"
    # Matches numbers with optional spaces, e.g., "4 000 Kč"

    # Apply regex to extract the fee
    for sentence in relevant_sentences:
        match = re.search(fee_pattern, sentence)
        if match:
            print("Extracted Fee:", match.group(1).replace(" ", ""))



In [4]:
for url, text in text_by_page.items():
    print(url)
    get_value_from_text(text)

http://skolkarybicka.cz
http://skolkarybicka.cz/
http://skolkarybicka.cz/program-skolky
http://skolkarybicka.cz/provoz
Extracted Fee: 5820
http://skolkarybicka.cz/fotogalerie
http://skolkarybicka.cz/kdo-jsme
http://skolkarybicka.cz/napsali-o-nas
http://skolkarybicka.cz/kontakt
http://skolkarybicka.cz/aktuality
http://skolkarybicka.cz/aktuality/zajmove-krouzky-a-aktivity-v-rybicce-leden-kveten-2025
http://skolkarybicka.cz/aktuality/prave-vypsany-zajmove-krouzky-v-rybicce
http://skolkarybicka.cz/aktuality/prvni-tridni-schuzka-3-zari-2024
Extracted Fee: 4
http://skolkarybicka.cz/pripravujeme
http://skolkarybicka.cz/podporuji-nas
http://skolkarybicka.cz/zajmove-krouzky-a-aktivity-v-rybicce
